# 4 Advanced Forecasting Pipelines

In the previous notebook, we considered
* sequential pipelines
* tuning their hyperparameters

This notebook is about:
* tuning the structure of sequential pipelines
* introducing graphical pipelines





In [1]:
import warnings
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler

from sktime.datasets import load_arrow_head, load_longley, load_macroeconomic, load_shampoo_sales
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    ColumnEnsembleForecaster,
    ForecastX,
    MultiplexForecaster,
    make_reduction,
)
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.trend import STLForecaster
from sktime.forecasting.theta import ThetaForecaster

from sktime.performance_metrics.forecasting import mean_absolute_error, MeanSquaredError
from sktime.pipeline.pipeline import Pipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.transformations.series.difference import Differencer
from sktime.transformations.series.exponent import ExponentTransformer
from sktime.transformations.series.subset import ColumnSelect
warnings.filterwarnings("ignore")

## 4.1 Tuning the Pipeline's Structure (AutoML)

* Until now, we only optimised the hyperparameters of the single models. 
* The pipeline structure choose influences the performance too.

`sktime` allows to expose these choices via structural compositors:

* switch between transform/forecast: `MultiplexTransformer`, `MultiplexForecaster`
* transformer on/off: `OptionalPassthrough`

Combine with pipelines and `FeatureUnion` for rich structure space

#### `MultiplexForecaster`
Switch between multiple forecasters. It is with a parameter `selected_forecaster: str` that can be tuned with a grid search.


<img src="img/multiplexer.png" width=900  style="background-color:white; padding:5px" />

In [2]:
from sktime.forecasting.compose import MultiplexForecaster

forecaster = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster()),
        ("stl", STLForecaster()),
        ("theta", ThetaForecaster()),
    ]
)


In [3]:
y = load_shampoo_sales()
y_train, y_test = temporal_train_test_split(y=y, test_size=6)
fh = ForecastingHorizon(y_test.index, is_relative=False).to_relative(
    cutoff=y_train.index[-1]
)

forecaster.set_params(selected_forecaster="stl")
forecaster.fit(y_train)
forecaster.predict(fh)

1993-07    400.583931
1993-08    463.705542
1993-09    414.450488
1993-10    477.572099
1993-11    428.317045
1993-12    491.438656
Freq: M, Name: Number of shampoo sales, dtype: float64

In [4]:
forecaster.set_params(selected_forecaster="naive")
forecaster.fit(y_train)
forecaster.predict(fh)

1993-07    437.4
1993-08    437.4
1993-09    437.4
1993-10    437.4
1993-11    437.4
1993-12    437.4
Freq: M, Name: Number of shampoo sales, dtype: float64

In [5]:
cv = SlidingWindowSplitter(fh=fh, window_length=len(y_train) - 6)

gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid={"selected_forecaster": ["naive", "stl", "theta"]},
    cv=cv,
    n_jobs=-1,
)

gscv.fit(y)
gscv.best_params_

{'selected_forecaster': 'theta'}

#### `Optional Passthrough`

Compositor that wraps a transformer and has a parameter `passthrough: bool`.
* Setting `passthrough=True` will return an identity transformation for the given data.
*  Setting `passthrough=False` will apply the given inner transformer on the data.

<img src="img/optional_passthrough.png" width=900  style="background-color:white; padding:5px" />


In [6]:
from sktime.transformations.compose import OptionalPassthrough

transformer = OptionalPassthrough(transformer=Detrender(), passthrough=True)
transformer.fit_transform(y_train).head()

Period
1991-01    266.0
1991-02    145.9
1991-03    183.1
1991-04    119.3
1991-05    180.3
Freq: M, Name: Number of shampoo sales, dtype: float64

In [7]:
y_train.head()

Period
1991-01    266.0
1991-02    145.9
1991-03    183.1
1991-04    119.3
1991-05    180.3
Freq: M, Name: Number of shampoo sales, dtype: float64

In [8]:
transformer = OptionalPassthrough(transformer=Detrender(), passthrough=False)
transformer.fit_transform(y_train).head()

Period
1991-01    130.376344
1991-02      1.503263
1991-03     29.930182
1991-04    -42.642900
1991-05      9.584019
Freq: M, Name: Number of shampoo sales, dtype: float64

## 4.1.1 AutoML and Forecasting

Taking all incredients from above examples, we can build a forecaster that comes close to what is usually called [AutoML](https://en.wikipedia.org/wiki/Automated_machine_learning).
With AutoML we aim to automate as many steps of an ML model creation as possible. The main compositions from `sktime` that we can use for this are:
- `TransformedTargetForecaster`
- `MultiplexForecaster`
- `ForecastingGridSearchCV`
- `OptionalPassthrough`

### Univariate example
Please see appendix section for an example with exogenous data

In [9]:
pipe_y = TransformedTargetForecaster(
    steps=[
        ("detrender", OptionalPassthrough(Detrender())),
        ("deseasonalizer", OptionalPassthrough(Deseasonalizer())),
        ("scaler", OptionalPassthrough(TabularToSeriesAdaptor(RobustScaler()))),
        ("forecaster",  MultiplexForecaster(forecasters=[
            ("naive", NaiveForecaster()),
            ("stl", STLForecaster()),
            ("theta", ThetaForecaster(deseasonalize=False)),
            ])),
    ]
)

param_grid = {
    "detrender__passthrough": [True, False],
    "deseasonalizer__passthrough": [True, False],
    "scaler__passthrough": [True, False],
    "scaler__transformer__transformer__with_scaling": [True, False],
    "scaler__transformer__transformer__with_centering": [True, False],
    "forecaster__selected_forecaster": ["naive", "stl", "theta"],
}

gscv = ForecastingGridSearchCV(
    forecaster=pipe_y,
    param_grid=param_grid,
    cv=cv,
    n_jobs=-1,
    verbose=1,
    scoring=MeanSquaredError(square_root=True),
    error_score="raise",
)

gscv.fit(y=y_train, fh=fh)

Fitting 1 folds for each of 96 candidates, totalling 96 fits


ForecastingGridSearchCV(cv=SlidingWindowSplitter(fh=ForecastingHorizon([1, 2, 3, 4, 5, 6], dtype='int32', is_relative=True),
                                                 window_length=24),
                        error_score='raise',
                        forecaster=TransformedTargetForecaster(steps=[('detrender',
                                                                       OptionalPassthrough(transformer=Detrender())),
                                                                      ('deseasonalizer',
                                                                       OptionalPassthrough(transformer=Deseasonalizer())),
                                                                      ('scaler',
                                                                       OptionalPas...
                        param_grid={'deseasonalizer__passthrough': [True,
                                                                    False],
                                    'detrender__passthrough': [True, False],
                                    'forecaster__selected_forecaster': ['naive',
                                                                        'stl',
                                                                        'theta'],
                                    'scaler__passthrough': [True, False],
                                    'scaler__transformer__transformer__with_centering': [True,
                                                                                         False],
                                    'scaler__transformer__transformer__with_scaling': [True,
                                                                                       False]},
                        scoring=MeanSquaredError(square_root=True), verbose=1)

In [10]:
gscv.best_params_

{'deseasonalizer__passthrough': True,
 'detrender__passthrough': False,
 'forecaster__selected_forecaster': 'naive',
 'scaler__passthrough': True,
 'scaler__transformer__transformer__with_centering': True,
 'scaler__transformer__transformer__with_scaling': True}

In [11]:
gscv.cv_results_["mean_test_MeanSquaredError"].min()

55.17836792311908

In [12]:
gscv.cv_results_["mean_test_MeanSquaredError"].max()

100.14534270723088

## 4.2 Graphical Pipeline

### What are Graphical Pipelines?
Recap sequential pipelines:

<img src="img/sequential_pipeline.png" width=900  style="background-color:white; padding:5px" />

Many tasks are non-sequential. To solve this two possibilities exist:
1. Nesting Sequential Pipelines.
2. Using Graphical Pipelines.


Generalised Graphical Pipeline in sktime:

* Graphical means that different steps may share the same predecessor or provide their outputs to the same successor (the dataflows can branch and merge).

<img src="img/graphical_pipeline.png" width=900  style="background-color:white; padding:5px" />


* Generalised means that the pipeline can be used for multiple tasks (e.g. forecasting, classification, ...).


**Note**

The graphical pipeline is still experimental. 
Thus, usual risk with bleeding edge features. 
However, we would be happy to get feedback on the graphical pipeline.


#### Forecasting Use-Case for Graphical Pipelines


The input of forecasters depends on the output of other forecasters, which same the same input.
* Forecaster could use the same preprocessing (branching of data flow)
* Forecaster could use outputs of multiple predeccessors (merging of data flow)

<img src="img/graphical_pipeline_example.png" width=900  style="background-color:white; padding:5px" />


### Credits
The graphical pipeline was first developed by pyWATTS [1] and was then adapted for sktime. The original implementation can be found [pyWATTS](https://github.com/KIT-IAI/pyWATTS). pyWATTS is a open source library developed at the Institute of Applied Informatics and Automation at the KIT and funded by HelmholtzAI.

> [1] Heidrich, Benedikt, et al. "pyWATTS: Python workflow automation tool for time series." arXiv preprint arXiv:2106.10157 (2021).

<img src="img/kit.png" height=60  style="background-color:white; padding:5px" /> 
<p></p>

<img src="img/helmholtz.png" width=900  style="background-color:white; padding:5px" />


### 4.2.1 Constructing simple Graphical Pipelines for Forecasting
<img src="img/forecasting_pipeline.png" width=900  style="background-color:white; padding:5px" />


Two ways to create graphical pipelines:

1. Pass all steps to the pipeline during initialisation as for the sequential pipeline.

In [13]:
differencer = Differencer()

pipe = Pipeline(
    [
        {"skobject": differencer, "name": "differencer", "edges": {"X": "y"}},
        {
            "skobject": ARIMA(),
            "name": "arima",
            "edges": {"X": "X", "y": "differencer"},
        },
        {
            "skobject": differencer,
            "name": "differencer_inv",
            "edges": {"X": "arima"},
            "method": "inverse_transform",
        },
    ]
)

**Note** if you add the same skobject instance multiple times, the graphical pipelines tracks the identity of these skobjects.

Alternatively, the pipeline can be also created using `add_step`

In [14]:
pipe = Pipeline()
differencer = Differencer()

pipe = pipe.add_step(
    differencer, "differencer", edges={"X": "y"}
)
pipe = pipe.add_step(
    ARIMA(), "arima", edges={"X": "X", "y": "differencer"}
)
pipe = pipe.add_step(
    differencer, "differencer_inv", edges={"X": "arima"}, method="inverse_transform"
)

#### Summary of the arguments:
The `add_step`'s parameter or key of the dicts in the step list during initialisation are:

* skobject: The sktime object added to the pipeline
* name: The name of the step
* edges: The keys of the dictionary indicate the input of the skobject (X or y), and the values are the names of the steps that should be connected to the input argument. Note subsetting using `__` and feature union via lists are supported.
* method: The skobject's method that should be called. If not provided, the default method would be inferred based on the added skobject. This parameter is used for the inverse_transform method. Optional.
* kwargs: Additional keyword arguments passed to the sktime object. Optional.

#### Take Away Messsage:
* Two ways to construct graphical pipeline
    * Provide all information during initialisation
    * Add each step separetely using `add_step` method

Fitting with graphical pipeline

In [15]:
y, X = load_longley()
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X)

pipe.fit(y=y_train, X=X_train, fh=[1, 2, 3, 4])


Pipeline(steps=[{'edges': {'X': 'y'}, 'kwargs': {}, 'method': None,
                 'name': 'differencer', 'skobject': Differencer()},
                {'edges': {'X': 'X', 'y': 'differencer'}, 'kwargs': {},
                 'method': None, 'name': 'arima', 'skobject': ARIMA()},
                {'edges': {'X': 'arima'}, 'kwargs': {},
                 'method': 'inverse_transform', 'name': 'differencer_inv',
                 'skobject': Differencer()}])

Predicting with graphical pipelines

In [16]:
pipe.predict(X=X_test)

1959    67213.735360
1960    68328.076304
1961    68737.861389
1962    71322.894013
Freq: A-DEC, Name: TOTEMP, dtype: float64

In [17]:
pipe.predict_quantiles(X=X_test)

TOTEMP              
              0.05          0.95
1959  66086.772692  68340.698028
1960  66067.640903  70588.511705
1961  65343.878044  72131.844734
1962  66795.361853  75850.426173

In [18]:
pipe.predict_interval(X=X_test)

TOTEMP              
               0.9              
             lower         upper
1959  66086.772692  68340.698028
1960  66067.640903  70588.511705
1961  65343.878044  72131.844734
1962  66795.361853  75850.426173

In [19]:
pipe.predict_residuals(X=X_test, y=y_test)

Period
1959    1441.264640
1960    1235.923696
1961     593.138611
1962    -771.894013
Freq: A-DEC, Name: TOTEMP, dtype: float64

### 4.2.2 A more complex example
The considered use-case is to forecast the inflation using forecasts of the real gross domestic product, real disposable personal income, and the unemployment rate. Furthermore the unemployment rate is forecasted using the same features except the unemployment rate itself.

<img src="img/graphical_pipeline_example.png" width=900 style="background-color:white; padding:5px" />


The data is taken from the macrodata dataset from the statsmodels package.

**Note** We stick with the `add_step` in the following.


Create Graphical Pipeline Instance

In [20]:
pipe = Pipeline()

Add Preprocessing

In [21]:
pipe = pipe.add_step(
    TabularToSeriesAdaptor(StandardScaler()), name="scaler", edges={"X": "X__realgdp_realdpi_unemp"}
)
pipe = pipe.add_step(
    Deseasonalizer(sp=4), name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

Add forecastesr for GDP and DPI

In [22]:
pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

Add Forecaster for unemployment rate that depends on forecasts of GDP and DPI

In [23]:
pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

Add forecaster for the inflation that depends on forecasted DPI and unemployment rate

In [24]:
pipe = pipe.add_step(
    make_reduction(Ridge(), windows_identical=False, window_length=5),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

Load data and split them into train and test

In [25]:
data = load_macroeconomic()

X = data[["realgdp", "realdpi", "unemp"]]
y = data[["infl"]]
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

y_train, y_test, X_train, X_test = temporal_train_test_split(y, X=X, fh=fh)
X_train

,realgdp,realdpi,unemp
Period,,,
1959Q1,2710.349,1886.9,5.8
1959Q2,2778.801,1919.7,5.1
1959Q3,2775.488,1916.4,5.3
1959Q4,2785.204,1931.3,5.6
1960Q1,2847.699,1955.5,5.2
...,...,...,...
2005Q3,12683.153,9308.0,5.0
2005Q4,12748.699,9358.7,4.9
2006Q1,12915.938,9533.8,4.7


In [26]:
pipe.fit(y=y_train, X=X_train, fh=fh)
result = pipe.predict(X=None, fh=y_test.index)
result

,infl
Period,
2006Q4,3.090428
2007Q1,1.676421
2007Q2,0.219586
2007Q3,1.570087
2007Q4,0.350137
2008Q1,0.438966
2008Q2,0.615457
2008Q3,0.119022
2008Q4,0.257887


#### But what are the best hyperparameters?

* which forecaster should be used for which variable -> `MultiplexForecaster`
* what should be the hyperparameters of the forecaster
* which features should be used for the different forecasters -> Tune the edges of the graphical pipeline!

**Idea:** Combine graphical pipeline with ForecastingGridSearchCV.


##### Simple Example

1. Define blueprint pipeline

In [27]:
differencer = Differencer()

pipe = Pipeline(
    [
        {"skobject": differencer, "name": "differencery", "edges": {"X": "y"}},
        {"skobject": Differencer(), "name": "differencerX", "edges": {"X": "X"}},
        {
            "skobject": MultiplexForecaster([
                ("arima", ARIMA()),
                ("ridge", make_reduction(Ridge(), windows_identical=False, window_length=3)),
                ("lasso", make_reduction(Lasso(), windows_identical=False, window_length=3))
            ]),
            "name": "forecaster",
            "edges": {"X": "differencerX", "y": "differencery"},
        },
        {
            "skobject": differencer,
            "name": "differencer_inv",
            "edges": {"X": "forecaster"},
            "method": "inverse_transform",
        },
    ]
)

2. Create parameter grid

The keys of the dictionary are the parameters' in the pipeline, and the values specify which options should be tested.
Keys have the following structure: parameter of a step `<step_name>__skobject__<parameter-name>` and input edges of a step `<step-name>__edges__<Xory>`.

In [28]:
param_grid = {
    "forecaster__skobject__selected_forecaster": ["ridge", "lasso", "arima"],
    "forecaster__edges__X": [
        [],
        ["differencerX"],
    ],
}

3. Initialise the gridsearch using pipeline, cross-validation strategy, scoring, and param_grid.


In [29]:
gridcv = ForecastingGridSearchCV(
    pipe,
    cv=SlidingWindowSplitter(
        window_length=len(X_train) - 4,
        step_length=4,
        fh=[1, 2, 3, 4],
    ),
    scoring=mean_absolute_error,
    param_grid=param_grid,
)

4. Perform Grid Search

In [30]:
gridcv.fit(y=y_train, X=X_train)

c:\Users\bi4372\PycharmProjects\sktime-workshop-pydata-prague-2023\venv\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ForecastingGridSearchCV(cv=SlidingWindowSplitter(fh=[1, 2, 3, 4], step_length=4,
                                                 window_length=187),
                        forecaster=Pipeline(steps=[{'edges': {'X': 'y'},
                                                    'kwargs': {},
                                                    'method': None,
                                                    'name': 'differencery',
                                                    'skobject': Differencer()},
                                                   {'edges': {'X': 'X'},
                                                    'kwargs': {},
                                                    'method': None,
                                                    'name': 'differencerX',
                                                    'skobject': Differencer()},
                                                   {'edges': {'X': 'differencerX',
                                                              'y': 'differen...
                                                                                                  RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                       window_length=3))])},
                                                   {'edges': {'X': 'forecaster'},
                                                    'kwargs': {},
                                                    'method': 'inverse_transform',
                                                    'name': 'differencer_inv',
                                                    'skobject': Differencer()}]),
                        param_grid={'forecaster__edges__X': [[],
                                                             ['differencerX']],
                                    'forecaster__skobject__selected_forecaster': ['ridge',
                                                                                  'lasso',
                                                                                  'arima']},
                        scoring=<function mean_absolute_error at 0x00000287E82D9430>)

In [31]:
gridcv.best_params_

{'forecaster__edges__X': [],
 'forecaster__skobject__selected_forecaster': 'ridge'}

#### Hyperparamter optimisation for complex example

<img src="img/graphical_pipeline_example_grid.png" width=900  style="background-color:white; padding:5px" />

1. Create blue print of the pipeline

In [32]:
pipe = Pipeline()
sklearn_scaler = StandardScaler()
sktime_scaler = TabularToSeriesAdaptor(sklearn_scaler)
deseasonalizer = Deseasonalizer(sp=4)

pipe = pipe.add_step(
    sktime_scaler, name="scaler", edges={"X": "X__realgdp_realdpi_unemp"}
)
pipe = pipe.add_step(
    deseasonalizer, name="deseasonalizer", edges={"X": "X__realgdp_realdpi"}
)

pipe = pipe.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_gdp",
    edges={"y": "deseasonalizer__realgdp"},
)

pipe = pipe.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_dpi",
    edges={"y": "deseasonalizer__realdpi"},
)

pipe = pipe.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_unemp",
    edges={
        "y": "scaler__unemp",
        "X": [
            "forecaster_gdp",
            "forecaster_dpi",
        ],
    },
)

pipe = pipe.add_step(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    name="forecaster_inflation",
    edges={"X": ["forecaster_dpi", "forecaster_unemp"], "y": "y"},
)

2. Specify Parameter Grid

In [33]:
param_grid = {
    "forecaster_inflation__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_unemp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_dpi__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_gdp__skobject__selected_forecaster": ["ridge", "lasso"],
    "forecaster_inflation__edges__X": [
        ["forecaster_unemp"],
        ["forecaster_unemp", "forecaster_dpi"],
    ],
    "forecaster_unemp__edges__X": [
        [],
        ["forecaster_dpi"],
        ["forecaster_gdp", "forecaster_dpi"],
    ],
    "deseasonalizer__edges__X": ["X__realgdp_realdpi", "scaler__realgdp_realdpi"],
}

3. Initialise the gridsearch using pipeline, cross-validation strategy, scoring, and param_grid.


In [34]:
gridcv = ForecastingGridSearchCV(
    pipe,
    cv=SlidingWindowSplitter(
        window_length=len(X_train) - 20,
        step_length=4,
        fh=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    ),
    scoring=mean_absolute_error,
    param_grid=param_grid,
    n_jobs=-1,
)

4. Call fit on the gridsearch object.


In [35]:
gridcv.fit(y=y_train, X=X_train)

ForecastingGridSearchCV(cv=SlidingWindowSplitter(fh=[1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                     10, 11, 12],
                                                 step_length=4,
                                                 window_length=171),
                        forecaster=Pipeline(steps=[{'edges': {'X': 'X__realgdp_realdpi_unemp'},
                                                    'kwargs': {},
                                                    'method': None,
                                                    'name': 'scaler',
                                                    'skobject': TabularToSeriesAdaptor(transformer=StandardScaler())},
                                                   {'edges': {'X': 'X__realgdp_realdpi'},
                                                    'kwargs': {},
                                                    'method': Non...
                                    'forecaster_inflation__edges__X': [['forecaster_unemp'],
                                                                       ['forecaster_unemp',
                                                                        'forecaster_dpi']],
                                    'forecaster_inflation__skobject__selected_forecaster': ['ridge',
                                                                                            'lasso'],
                                    'forecaster_unemp__edges__X': [[],
                                                                   ['forecaster_dpi'],
                                                                   ['forecaster_gdp',
                                                                    'forecaster_dpi']],
                                    'forecaster_unemp__skobject__selected_forecaster': ['ridge',
                                                                                        'lasso']},
                        scoring=<function mean_absolute_error at 0x00000287E82D9430>)

In [36]:
gridcv.cv_results_

,mean_test__DynamicForecastingErrorMetric,mean_fit_time,mean_pred_time,params,rank_test__DynamicForecastingErrorMetric
0,1.539329,0.220413,0.128662,{'deseasonalizer__edges__X': 'X__realgdp_reald...,107.5
1,1.720565,0.156704,0.075365,{'deseasonalizer__edges__X': 'X__realgdp_reald...,119.5
2,1.394329,0.435990,0.219232,{'deseasonalizer__edges__X': 'X__realgdp_reald...,97.5
3,1.942051,0.445285,0.230354,{'deseasonalizer__edges__X': 'X__realgdp_reald...,129.5
4,2.033714,0.318611,0.172922,{'deseasonalizer__edges__X': 'X__realgdp_reald...,136.0
...,...,...,...,...,...
187,1.329079,0.391476,0.149731,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
188,1.329079,0.253248,0.142579,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
189,1.329079,0.290770,0.131217,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5
190,1.329079,0.292910,0.170033,{'deseasonalizer__edges__X': 'scaler__realgdp_...,48.5


In [37]:
gridcv.best_params_

{'deseasonalizer__edges__X': 'X__realgdp_realdpi',
 'forecaster_dpi__skobject__selected_forecaster': 'ridge',
 'forecaster_gdp__skobject__selected_forecaster': 'ridge',
 'forecaster_inflation__edges__X': ['forecaster_unemp', 'forecaster_dpi'],
 'forecaster_inflation__skobject__selected_forecaster': 'lasso',
 'forecaster_unemp__edges__X': [],
 'forecaster_unemp__skobject__selected_forecaster': 'ridge'}

<img src="img/graphical_pipeline_example_grid_best_params.png" width=900  style="background-color:white; padding:5px" />


In [38]:
result = gridcv.predict(X=None, fh=y_test.index)
result

,infl
Period,
2006Q4,2.188182
2007Q1,2.124281
2007Q2,1.045280
2007Q3,1.857716
2007Q4,1.790664
2008Q1,1.649457
2008Q2,1.874361
2008Q3,1.855627
2008Q4,1.858207


### 4.2.3 How does the graphical pipeline compare to the sequential pipeline?

Let us try to implement a simplified version of the above example using sequential pipelines with nesting.

<img src="img/graphical_pipeline_simplified.png" width=900  style="background-color:white; padding:5px" />


Create sequential pipelines for forecasting the GDP, DPI and unemployment rate.


In [39]:
forecasting_pipeline_gdp = (
    ColumnSelect(["realgdp"])  # To train the forecaster only on the realgdp column
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)
forecasting_pipeline_dpi = (
    ColumnSelect(["realdpi"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

forecasting_pipeline_unemp = (
    ColumnSelect(["unemp"])
    * Deseasonalizer()
    * MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    )
)

Use ColunmEnsembleForecaster to combine the forecasts of the DPI, GDP, UNEMP. (Union of forecasts)

In [40]:
input_inflation_forecast = ColumnEnsembleForecaster(
    [
        ("realdpi", forecasting_pipeline_dpi, "realdpi"),
        ("realgdp", forecasting_pipeline_gdp, "realgdp"),
        ("unemp", forecasting_pipeline_unemp, "unemp"),
    ]
)

Create the inflation forecaster.

In [41]:
inflation_forecast = ForecastX(
    MultiplexForecaster(
        [
            (
                "ridge",
                make_reduction(Ridge(), windows_identical=False, window_length=5),
            ),
            (
                "lasso",
                make_reduction(Lasso(), windows_identical=False, window_length=5),
            ),
        ]
    ),
    input_inflation_forecast,
)

In [42]:
inflation_forecast.fit(y=y_train, X=X_train, fh=fh)

ForecastX(forecaster_X=ColumnEnsembleForecaster(forecasters=[('realdpi',
                                                              TransformedTargetForecaster(steps=[ColumnSelect(columns=['realdpi']),
                                                                                                 Deseasonalizer(),
                                                                                                 MultiplexForecaster(forecasters=[('ridge',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                        window_length=5)),
                                                                                                                                  ('lasso',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                                                        window_length...
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                                                                                                        window_length=5)),
                                                                                                                                  ('lasso',
                                                                                                                                   RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                                                                                                        window_length=5))])]),
                                                              'unemp')]),
          forecaster_y=MultiplexForecaster(forecasters=[('ridge',
                                                         RecursiveTabularRegressionForecaster(estimator=Ridge(),
                                                                                              window_length=5)),
                                                        ('lasso',
                                                         RecursiveTabularRegressionForecaster(estimator=Lasso(),
                                                                                              window_length=5))]))

In [43]:
inflation_forecast.predict()

,infl
2006Q4,3.979318
2007Q1,2.347512
2007Q2,1.443598
2007Q3,3.914533
2007Q4,2.533117
2008Q1,3.278010
2008Q2,3.861517
2008Q3,3.487510
2008Q4,4.195074
2009Q1,4.294984


Ok, we can built it, but what about the hyperparameter tuning? Let us investigate the paramters..

In [44]:
list(inflation_forecast.get_params(True).keys())

['behaviour',
 'columns',
 'fh_X',
 'forecaster_X',
 'forecaster_y',
 'forecaster_X__forecasters',
 'forecaster_X__realdpi',
 'forecaster_X__realgdp',
 'forecaster_X__unemp',
 'forecaster_X__realdpi__steps',
 'forecaster_X__realdpi__ColumnSelect',
 'forecaster_X__realdpi__Deseasonalizer',
 'forecaster_X__realdpi__MultiplexForecaster',
 'forecaster_X__realdpi__ColumnSelect__columns',
 'forecaster_X__realdpi__ColumnSelect__index_treatment',
 'forecaster_X__realdpi__ColumnSelect__integer_treatment',
 'forecaster_X__realdpi__Deseasonalizer__model',
 'forecaster_X__realdpi__Deseasonalizer__sp',
 'forecaster_X__realdpi__MultiplexForecaster__forecasters',
 'forecaster_X__realdpi__MultiplexForecaster__selected_forecaster',
 'forecaster_X__realdpi__MultiplexForecaster__ridge',
 'forecaster_X__realdpi__MultiplexForecaster__lasso',
 'forecaster_X__realdpi__MultiplexForecaster__ridge__estimator',
 'forecaster_X__realdpi__MultiplexForecaster__ridge__pooling',
 'forecaster_X__realdpi__MultiplexForec

# Augh...


### Advantages of sequential pipelines
* Constructing simple pipelines is very easy.
* Inverse operations are automatically applied.
* This is a mature feature compared to the experimental graphical pipeline.


### Advantages of graphical pipelines
* Enable an easy implementation of complex pipelines
    * By nesting sequential pipelines, even a simplified version of the graphical pipeline is very complicat to implement.
    * By nesting sequential pipelines, some graphical pipelines are not possible to implement (e.g., the example with coupled ForecastX).
* Preprocessing steps can not be shared between the different forecasters.
* The parameter structure is simpler compared to sequential pipelines. 
    * Thus easier to tune the structure also in complex secnarios. How would you tune the edges of sequential pipelines?
* Only one estimator to track compared to multiples in the sequential pipeline example.



## Outlook

Graphical Pipelines are a powerful tool.

You are invited to contribute and to make it production-ready.
* Enhance User Experience:
    * Enable a drawing of the graphical pipeline
    * Add more examples
* Apply Graphical Pipelines in Research Topics
    * AutoML for optimising forecast value and not only the forecast quality.
    * Concept Drift: What parts of the graphical pipelines you want to retrain?
* Improve Graphical Pipelines Performance
    * Parallelize steps!
    * Optimise GridSearch for Graphical Pipelines
* Use it and provide feedback

---

### Credits: notebook 4 - advanced 

notebook creation: benheid

autoML based on:

* pyData global 2022 notebook (aiwalter, fkiraly)
* notebooks at ODSC and EuroPython 2023 workshops (fkiraly, marrov)

graphical pipeline: benheid, fkiraly, pywatts team\
forecaster pipelines: fkiraly, aiwalter\
transformer pipelines & compositors: fkiraly, mloning, miraep8\
dunder interface: fkiraly, miraep8\
tuning, autoML: mloning, fkiraly, aiwalter